# Seq2Seq: Translation Chatbot with Attention
#### アテンションLSTMによる機械翻訳 

![](https://cdn-images-1.medium.com/max/2600/1*1I2tTjCkMHlQ-r73eRn4ZQ.png)

## Encoder-Decoderアーキテクチャ
1. Sentence Data を3つのNumpy配列、（encoder_input_data、decoder_input_data、decoder_target_data）に変換.
  - encoder_input_dataは、（num_pairs、max_english_sentence_length、num_english_characters）の3D配列. *英語文はOne-Hotベクトル
  - decoder_input_dataは、（num_pairs、max_french_sentence_length、num_french_characters）の3D配列. *日本語文はOne-Hotベクトル
  - decoder_target_dataは、decoder_input_dataと同じだが、1 Time Step オフセットされている.
2. encoder_input_dataとdecoder_input_dataが与えられ、decoder_target_dataを予測するために基本的なLSTMベースのSeq2Seqモデルを訓練する.
3. いくつかのサンプル文をデコードして、モデルが機能していることを確認する.

In [1]:
ls

0315_seq2seq_translater.ipynb  seq2seq_translater.ipynb
jpn.txt


In [2]:
import numpy as np
import pandas as pd
import string
import pickle
import operator
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Preprocess feeding data

### 2-1. 日英会話データをロード

In [3]:
with open("jpn.txt", 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [4]:
len(lines)

43954

In [5]:
lines[0]

'Go.\t行け。'

In [6]:
input_texts = []
target_texts = []
bos = "<BOS> "
eos = " <EOS>"

for line in lines:
    if len(line.split("\t")) == 2:
        input_text, target_text = line.split("\t")[0], line.split("\t")[1]
        target_text = bos + target_text + eos
        input_texts.append(input_text)
        target_texts.append(target_text)

len(input_texts)

43953

In [7]:
num_samples = len(target_texts)
num_samples

43953

In [8]:
input_texts[40000]

'I had hardly opened my mouth, when she interrupted me.'

In [9]:
target_texts[40000]

'<BOS> まだ私がほとんど何も言わないうちに彼女が割って入った。 <EOS>'

### 2-2. トークン化

In [10]:
# 英語をcharacter単位で分解
input_characters = set()

for text in input_texts:
    char = text.split()
    for char in input_texts:
        if char not in input_characters:
            input_characters.add(char)

input_characters = sorted(list(input_characters))
english_char_size = len(input_characters)
english_char_size

37636

In [11]:
# 日本語をcharacter単位で分解
target_characters = set()

for text in target_texts:
    char = text.split()
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

target_characters = sorted(list(target_characters))
japanese_char_size = len(target_characters)
japanese_char_size

49

In [12]:
# 最大文章の長さをえる
english_maxlen = max([ len(text) for text in input_texts ])
japanese_maxlen = max([ len(text) for text in target_texts ])

### 2-3. ベクトル化&辞書作り

In [13]:
input_char_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])

In [14]:
target_char_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [15]:
input_index_char = dict(
    (i, char) for char, i in input_char_index.items())

In [16]:
target_index_char = dict(
    (i, char) for char, i in target_char_index.items())

In [17]:
# 3Dテンソル (サンプル数, MAX_LEN, 辞書サイズ)
encoder_input_data = np.zeros(
    (len(input_texts), english_maxlen, english_char_size),
    dtype="float32")

decoder_input_data = np.zeros(
    (len(input_texts), japanese_maxlen, japanese_char_size),
    dtype="float32")

decoder_output_data = np.zeros(
    (len(input_texts), japanese_maxlen, japanese_char_size),
    dtype="float32")

In [21]:
input_texts[:10]

['Go.', 'Go.', 'Hi.', 'Hi.', 'Run.', 'Run.', 'Who?', 'Wow!', 'Wow!', 'Wow!']

In [ ]:
input_char_index

In [30]:
for i, text in enumerate(input_texts):
    char = char.lower()
    for j, char in enumerate(text):
        encoder_input_data[i][j][input_char_index[char]] = 1.

KeyError: 'G'

In [31]:
for i, text in enumerate(target_texts):
    for j, char in enumerate(text):
        decoder_input_data[i][j][target_char_index[char]] = 1.
        # decoderのアウトプットは1ステップずれる
        if j > 0:
            decoder_output_data[i, j-1, target_char_index[char]] = 1.

KeyError: 'け'

## 2. Modeling Seq2Seq

## Process
1. Encode the input sequence into state vectors.
2. Start with a target sequence of size 1 (just the start-of-sequence character).
3. Feed the state vectors and 1-char target sequence to the decoder to produce predictions for the next character.
4. Sample the next character using these predictions (we simply use argmax).
5. Append the sampled character to the target sequence
6. Repeat until we generate the end-of-sequence character or we hit the character limit.

- Encoder入力データ: id化された単語シーケンス（英語）
- Encoder入力層: None×単語シーケンスの長さの2次元マトリックス
- EncoderのLSTM層: 指定の隠れ層のユニット数
- Decoder入力データ: id化された単語シーケンス（日本語）
- Decoder入力層: None×単語シーケンスの長さの2次元マトリックス
- DecoderのLSTM層: 指定の隠れ層のユニット数

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [ ]:
HIDDEN_DIM = 256
NUM_ENCODER_TOKENS = english_char_size
NUM_DECODER_TOKENS = japanese_char_size

In [ ]:
encoder_inputs = Input(shape=(None, NUM_ENCODER_TOKENS))
encoder_LSTM = LSTM(units=HIDDEN_DIM, return_state=True) # this is the key
encoder_outputs, state_h, state_c = encoder_LSTM(encoder_inputs) # encoderの出力は無視する、memory cellの状態だけ保存
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, NUM_DECODER_TOKENS))
decoder_LSTM = LSTM(units=HIDDEN_DIM, return_sequences=True, return_state=True) # this is the key
decoder_outputs, _h, _c = decoder_LSTM(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(units=NUM_DECODER_TOKENS, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [ ]:
model.summary()

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file = "seq2seq_translation.png", show_shapes = True)

## LSTM Layer works as encoder and decoder
- Encoder: 入力シーケンスを処理し、それ自体の内部状態を返す.この内部状態は、次のステップでDecoderの「文脈」と「状態」として機能する.
- Decoder: 1つ前の文字があれば、ターゲットシーケンスの次の文字を予測するように訓練されている. 
- Thought Vector: Encoderは初期状態としてEncoderからの状態ベクトルを使用. これはDecoderが生成するものに関する情報を取得するメソッド.

- return_stateコンストラクタ引数。
- 最初のエントリが出力で次のエントリが内部RNN状態であるリストを返すようにRNNレイヤを設定します。
- これはエンコーダの状態を回復するために使用されます。
- RNNの初期状態を指定するinital_state呼び出し引数。
- これは、初期状態としてエンコーダ状態をデコーダに渡すために使用されます。
- return_sequencesコンストラクター引数。
- RNNがその完全な出力シーケンスを返すように設定します（デフォルトの動作である最後の出力だけではなく）。
- これはデコーダで使用されます。

## 3. Fitting and Visualization

In [ ]:
BATCH_SIZE = 32
EPOCHS = 5

In [ ]:
model.fit([encoder_input_data, decoder_input_data], decoder_output_data,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.2)

model.save("seq2seq_translation.h5")

In [ ]:
BATCH_SIZE = [32, 64]
EPOCHS = [5, 10]

for i in range(2):
    BATCH_SIZE = BATCH_SIZE[i]
    for e in range(2):
        EPOCHS = EPOCHS[e]
        print("------------------------------------")
        print("BATCH_SIZE: ", BATCH_SIZE)
        print("EPOCHS: ", EPOCHS)
        history = model.fit([encoder_input_data, decoder_input_data], decoder_output_data,
                            batch_size=BATCH_SIZE,
                            epochs=EPOCHS,
                            validation_split=0.2)

## 4. Inference

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(HIDDEN_DIM,))
decoder_state_input_c = Input(shape=(HIDDEN_DIM,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, _h, _c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [_h, _c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [ ]:
def inference_translater(english_input):
    
    encoder_states = encoder_model.predict(english_input)

    japanese_seq = np.zeros((1, 1, NUM_DECODER_TOKENS))
    japanese_seq[0, 0, target_char_index[bos]] = 1.
    
    stop_condition = False
    japanese_output = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([japanese_seq] + encoder_states)
        # ソフトマックスが最大のcharのidを返す np.argmax
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        # 日本語の単語に変換
        sampled_char = target_index_char[sampled_token_index]
        japanese_output += sampled_char

        # Exit condition: either hit max length or find stop character.
        if (sampled_char == eos or len(japanese_output) > japanese_maxlen):
            stop_condition = True

        japanese_seq = np.zeros((1, 1, NUM_DECODER_TOKENS))
        japanese_seq[0, 0, sampled_token_index] = 1.

        encoder_states = [_h, _c]

    return japanese_output

In [ ]:
english_text = "I am so tired and I feel bored to go back to my home."

In [ ]:
inference_translater(english_text)